# 案例 2.1: 承压输水管网闭环控制

本案例演示了一个承压管网的闭环控制系统。一个智能体 (`PressureControlAgent`) 负责监控管道末端的压力，并通过控制上游的一个泵站来将压力维持在目标区间内。同时，一个下游的阀门会模拟不断变化的用户用水需求。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# 导入平台组件
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.simulation_identification.physical_objects.valve import Valve
from swp.simulation_identification.physical_objects.pump import Pump, PumpStation
from swp.local_agents.control.pressure_control_agent import PressureControlAgent

CONFIG_PATH = 'examples/pressurized_network_control.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义仿真函数

In [ ]:
def run_pressurized_control_simulation(cfg):
    # 1. 初始化
    bus = MessageBus()
    sim_cfg = cfg['simulation']
    dt = sim_cfg['dt']
    comp_cfgs = cfg['components']
    agent_cfg = cfg['agent_config']

    # 2. 创建物理组件
    res_cfg = comp_cfgs['upstream_reservoir']
    upstream_reservoir = Reservoir(name="source_res", initial_state={'water_level': res_cfg['initial_water_level']}, parameters={})

    ps_cfg = comp_cfgs['pump_station']
    pumps = [Pump(f"p_{i}", {'status': 0}, ps_cfg['pump_params'], bus, f"{agent_cfg['control_topic_prefix']}.p_{i}") for i in range(ps_cfg['num_pumps'])]
    pump_station = PumpStation(name="main_ps", initial_state={}, parameters={}, pumps=pumps)

    pipe = Pipe(name="main_pipe", initial_state={}, parameters=comp_cfgs['pipe'])

    valve_cfg = comp_cfgs['demand_valve']
    demand_valve = Valve(name="demand_valve", initial_state={'opening': valve_cfg['initial_opening']}, parameters=valve_cfg)

    # 3. 创建控制智能体
    agent = PressureControlAgent(
        agent_id="PressureRegulator",
        message_bus=bus,
        pump_station=pump_station,
        pressure_source_component=pipe,
        control_topic_prefix=agent_cfg['control_topic_prefix'],
        config={'min_pressure': agent_cfg['min_pressure'], 'max_pressure': agent_cfg['max_pressure']}
    )

    # 4. 仿真循环
    print("\n--- 开始仿真 ---")
    print(f"控制目标: 维持压力在 {agent_cfg['min_pressure']}m 和 {agent_cfg['max_pressure']}m 之间。\n")
    history = []
    demand_events = {event['time_step']: event for event in cfg['demand_changes']}

    for t in range(sim_cfg['num_steps']):
        # 模拟需求变化
        if t in demand_events:
            opening = demand_events[t]['opening']
            print(f"\n>>> 需求变化: 下游阀门开度变为 {opening}% <<<")
            demand_valve.step({'control_signal': opening}, dt)

        # 智能体决策
        agent.execute_control_logic()

        # 系统物理状态更新（使用修正后的泵驱动系统逻辑）
        num_active_pumps = agent.active_pumps
        pump_head_gain = ps_cfg['pump_params']['max_head'] if num_active_pumps > 0 else 0
        pipe_inlet_head = upstream_reservoir.get_state()['water_level'] + pump_head_gain
        
        # 泵决定的流量
        pump_station_flow = num_active_pumps * ps_cfg['pump_params']['max_flow_rate']
        pipe.set_inflow(pump_station_flow)
        pipe.step({}, dt) # 计算该流量下的水头损失
        pipe_head_loss = pipe.get_state()['head_loss']

        # 管道末端压力 = 入口压力 - 损失
        final_pipe_outlet_head = pipe_inlet_head - pipe_head_loss
        pipe.set_state({'downstream_head': final_pipe_outlet_head})

        # 记录日志
        log_entry = {
            "time": t * dt,
            "valve_opening": demand_valve.get_state()['opening'],
            "active_pumps": agent.active_pumps,
            "flow": pump_station_flow,
            "pressure": final_pipe_outlet_head
        }
        history.append(log_entry)
        print(f"时间: {log_entry['time']:3d}s | 运行水泵数: {log_entry['active_pumps']} | 流量: {log_entry['flow']:.2f} | 压力: {log_entry['pressure']:.2f}m")
        
    return pd.DataFrame(history)

### 3. 运行仿真并可视化结果

In [ ]:
results_df = run_pressurized_control_simulation(config)

fig, ax1 = plt.subplots(figsize=(15, 7))
fig.suptitle('承压管网压力控制仿真结果', fontsize=16)

ax2 = ax1.twinx() # 创建共享x轴的双y轴

# 绘制压力曲线 (ax1)
ax1.plot(results_df['time'], results_df['pressure'], 'b-', label='管道末端压力 (m)')
ax1.axhline(y=config['agent_config']['min_pressure'], color='r', linestyle=':', label='最低压力阈值')
ax1.axhline(y=config['agent_config']['max_pressure'], color='r', linestyle=':', label='最高压力阈值')
ax1.set_xlabel('时间 (s)')
ax1.set_ylabel('压力 (m)', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True)

# 绘制运行水泵数量 (ax2)
ax2.step(results_df['time'], results_df['active_pumps'], 'g-', where='post', label='运行水泵数量')
ax2.set_ylabel('运行水泵数量', color='g')
ax2.tick_params(axis='y', labelcolor='g')
ax2.set_ylim(-0.5, config['components']['pump_station']['num_pumps'] + 0.5)

# 合并图例
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.show()